In [8]:
from pathlib import Path

import numpy as np
import pandas as pd
import xarray as xr

import matplotlib.pyplot as plt

from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error, mean_absolute_error

from spicy_snow.retrieval import retrieval_from_parameters

# import functions for downloading
from spicy_snow.download.sentinel1 import s1_img_search, hyp3_pipeline, download_hyp3, combine_s1_images
from spicy_snow.download.forest_cover import download_fcf
from spicy_snow.download.snow_cover import download_snow_cover

# import functions for pre-processing
from spicy_snow.processing.s1_preprocessing import merge_partial_s1_images, s1_orbit_averaging,\
s1_clip_outliers, subset_s1_images, ims_water_mask, s1_incidence_angle_masking, merge_s1_subsets

# import the functions for snow_index calculation
from spicy_snow.processing.snow_index import calc_delta_VV, calc_delta_cross_ratio, \
    calc_delta_gamma, clip_delta_gamma_outlier, calc_snow_index, calc_snow_index_to_snow_depth

# import the functions for wet snow flag
from spicy_snow.processing.wet_snow import id_newly_frozen_snow, id_newly_wet_snow, \
    id_wet_negative_si, flag_wet_snow

# setup root logger
from spicy_snow.utils.spicy_logging import setup_logging

# fischer z test
from causallearn.utils.cit import CIT

# random fields
from gstools import SRF
from gstools import vario_estimate_unstructured
from gstools import Exponential

In [78]:
def s1_to_sd(ds, A = 1.5, B = 0.1, C = 0.6):
    # mask out outliers in incidence angle
    ds = s1_incidence_angle_masking(ds)
    
    # subset dataset by flight_dir and platform
    dict_ds = subset_s1_images(ds)

    for subset_name, subset_ds in dict_ds.items():
        # average each orbit to overall mean
        dict_ds[subset_name] = s1_orbit_averaging(subset_ds)
        # clip outlier values of backscatter to overall mean
        dict_ds[subset_name] = s1_clip_outliers(subset_ds)
    
    # recombine subsets
    ds = merge_s1_subsets(dict_ds)

    # calculate confidence interval
    # ds = add_confidence_angle(ds)

    ## Snow Index Steps
    # log.info("Calculating snow index")
    # calculate delta CR and delta VV
    ds = calc_delta_cross_ratio(ds, A = A)
    ds = calc_delta_VV(ds)

    # calculate delta gamma with delta CR and delta VV with FCF
    ds = calc_delta_gamma(ds, B = B)

    # clip outliers of delta gamma
    ds = clip_delta_gamma_outlier(ds)

    # calculate snow_index from delta_gamma
    ds = calc_snow_index(ds, ims_masking = True)

    # convert snow index to snow depth
    ds = calc_snow_index_to_snow_depth(ds, C = C)

    return ds

import warnings
def get_gaussian_stats(da):
    arr = da.values
    warnings.filterwarnings("ignore", message="Mean of empty slice")
    warnings.filterwarnings("ignore", message="Degrees of freedom <= 0 ")
    return arr.size, np.nanmean(arr), np.nanstd(arr), arr.shape

def bias(x, y): return np.mean(x - y)

def get_stats(x, y, nrmse = False):
    if type(x) == xr.DataArray: x = x.values.ravel()
    if type(y) == xr.DataArray: y = y.values.ravel()
    if type(x) == list: x = np.array(x)
    if type(y) == list: y = np.array(y)
    idx = (~np.isnan(x)) & (~np.isnan(y))
    x, y = x[idx], y[idx]
    r, p = pearsonr(x, y)
    b = bias(x, y)
    mae = mean_absolute_error(x, y)
    rmse = mean_squared_error(x, y, squared = False)

    if nrmse:
        nrmse_value = rmse / np.mean(x)
        return r, b, mae, rmse, nrmse_value

    return r, b, mae, rmse

from scipy.stats import norm
def fischerz(truth, x1, x2):
    idx = (~np.isnan(truth)) & (~np.isnan(x1)) & (~np.isnan(x2))
    truth, x1, x2 = truth[idx], x1[idx], x2[idx]
    n = len(x1)
    cor1 = pearsonr(truth, x1).statistic
    cor2 = pearsonr(truth, x2).statistic
    fischer1 = 0.5*np.log((1+cor1)/(1-cor1))
    fischer2 = 0.5*np.log((1+cor2)/(1-cor2))
    expected_sd = np.sqrt(1/(n-3))
    return 2 * (1 - norm(0, expected_sd).cdf(np.abs(fischer1 - fischer2)))

def optimize(r_ds, A, B, C, im_date):
    # holds rmse and r for optimizing rand dataset
    rand_rmse_ds = xr.DataArray(np.zeros([len(A), len(B), len(C)]), dims = ['A', 'B', 'C'], coords = [A, B, C])
    rand_mae_ds = xr.DataArray(np.zeros([len(A), len(B), len(C)]), dims = ['A', 'B', 'C'], coords = [A, B, C])
    rand_r_ds = xr.DataArray(np.zeros([len(A), len(B), len(C)]), dims = ['A', 'B', 'C'], coords = [A, B, C])

    # optimize random dataset
    for a in A:
        for b in B:
            for c in C:
                r_ds = s1_to_sd(r_ds, A = a, B = b, C = c)
                rand_r, rand_b, rand_mae, rand_rmse = get_stats(r_ds['snow_depth'].sel(time = im_date, method = 'nearest'), r_ds['lidar-sd'], nrmse = False)
                rand_rmse_ds.loc[dict(A = a, B = b, C = C)] = rand_rmse
                rand_mae_ds.loc[dict(A = a, B = b, C = C)] = rand_mae
                rand_r_ds.loc[dict(A = a, B = b, C = C)] = rand_r
    
    return rand_rmse_ds, rand_mae_ds, rand_r_ds

def create_unstructured_grid(x_s, y_s):
    x_u, y_u = np.meshgrid(x_s, y_s)
    len_unstruct = len(x_s) * len(y_s)
    x_u = np.reshape(x_u, len_unstruct)
    y_u = np.reshape(y_u, len_unstruct)
    return x_u, y_u

def create_random_field(da):
    random_ds = da.copy(deep = True)
    arr = da.data
    x_u, y_u = create_unstructured_grid(da.x.data, da.y.data)
    bins = np.linspace(20, 1500, 20)
    bin_center, gamma = vario_estimate_unstructured(
            (x_u, y_u), arr.flatten(), bins, sampling_size=3000)
    fit_model = Exponential(dim=2, nugget= 0)
    fit_model.fit_variogram(bin_center, gamma, nugget=True)
    srf = SRF(fit_model, seed=19770928)
    # this transpose appears in all the documentation and is neccessary to make the data fit the original data
    random_ds.data = srf((da.x.data, da.y.data), mesh_type='structured').T

    return random_ds

def make_random_timeseries(ds):
    """
    Makes a random time series of s1 data with t0 VV having the same spatial correlation length and variances
    as the real data and VH @ t = 0 being a mean shifted version of the random VV @ t0. Then the difference from 
    t = 0 to t = n is calculated for eachtime step and a change with the same varaince and spatial correlation length
    is added to backscatter to t = 0.
    """
    ds = ds.rio.write_crs('EPSG:4326')
    random_ds = ds[['s1','fcf', 'ims']].copy(deep = True)

    for orbit_name, sub in ds.groupby('relative_orbit'):

        for band in ['VV','VH']:
            sub[band] = sub['s1'].sel(band = band)
        sub = sub.drop('s1')
        sub = sub.drop('band')
        sub = sub.transpose('time', 'y', 'x')
        sub = sub.rio.reproject(sub.rio.estimate_utm_crs())
        
        
        random_sub_ds = sub.copy(deep = True)
        
        # make a random visualization of VV @ t = 0 with matching variance and correlation length to real data
        random_sub_ds['VV'].loc[dict(time = sub.isel(time = 0).time)] = create_random_field(sub['VV'].isel(time = 0))

        # make VH @ t = 0 match VV with a shifted mean (arbitrary since we just use change in backscatter) to simulate the geometric drive correlation between the two 
        random_sub_ds['VH'].loc[dict(time = sub.isel(time = 0).time)] =  random_sub_ds['VV'].loc[dict(time = sub.isel(time = 0).time)] - 1
        
        sub['deltaVH'] = sub['VH'].diff(dim = 'time')

        for i, (ts, time_ds) in enumerate(sub.groupby('time')):
            if i == 0:
                continue
            # go through each time series with dVV and then dVH
            # add random field with matching spatial statistics to dVV and dVH to original image to make new timeseries
            for band in ['VV', 'VH']:
                random_sub_ds[band].loc[dict(time = ts)] = random_sub_ds[band].isel(time = 0) + create_random_field(time_ds[band] - random_sub_ds[band].isel(time = 0))
        
        
        for band in ['VV', 'VH']:
            random_ds['s1'].loc[dict(time = ds.relative_orbit == orbit_name, band = band)] = random_sub_ds[band].rio.reproject_match(random_ds['s1'].loc[dict(time = ds.relative_orbit == orbit_name, band = band)])

    return random_ds

In [79]:
in_dir = Path('~/scratch/spicy/SnowEx-Data/').expanduser().resolve()
data_dir = Path('~/scratch/spicy/SnowEx-Data/').expanduser().resolve()
out_dir = Path('/bsuhome/zacharykeskinen/spicy-analysis/results/synthetic_compare')
dss = {fp.stem: xr.open_dataset(fp) for fp in in_dir.glob('*.nc')}

# Create parameter space
A = np.round(np.arange(1, 3.1, 0.5), 2)
B = np.round(np.arange(0, 2.01, 0.25), 2)
C = np.round(np.arange(0.25, 1.001, 0.25), 2)

# random field starting with matching VV and VH (with a contant offset) from the spatial statistics of VV @ t =0 and then iteratively add a random field calculated from that time steps dVV and dVH

In [101]:
if out_dir.joinpath('synthetic_v1.csv').exists():
    res = pd.read_csv(out_dir.joinpath('synthetic_v1.csv'), index_col=0)
else:
    res = pd.DataFrame()
rands =[]
s1 = []
sd = []

# Create parameter space
A = np.round(np.arange(1, 3.1, 0.5), 2)
B = np.round(np.arange(0, 2.01, 0.25), 2)
C = np.round(np.arange(0.1, 1.001, 0.2), 2)

for stem, full_ds in dss.items():
    full_ds = full_ds.load()
    site_name = stem.replace('_', ' ').replace('Frasier', 'Fraser').split('-')[0]
    
    if site_name in res.index:
        pass
    
    # if stem != 'Little_Cottonwood_2021-03-18':
    #     continue
    # full_ds = full_ds.load().isel(time = slice(50, None))

    if stem == 'Frasier_2020-02-11':
        im_date = pd.to_datetime('2020-02-16')
    else:
        im_date = pd.to_datetime(full_ds.sel(time = full_ds.attrs['lidar-flight-time'], method = 'nearest').time.values.ravel()[0])
    
    random_ds = make_random_timeseries(full_ds)

    # print values
    print(site_name + ' random created')
    
    random_ds['lidar-sd'] = full_ds['lidar-sd']
    rand_rmse_ds, rand_mae_ds, rand_r_ds = optimize(random_ds, A, B, C, im_date)

    a_best = rand_r_ds.max(['B', 'C']).idxmax('A')
    b_best = rand_r_ds.max(['C', 'A']).idxmax('B')
    c_best = rand_mae_ds.sel(A = a_best, B = b_best).idxmin('C')
    
    r_ds = s1_to_sd(random_ds, A = a_best, B = b_best, C = c_best).sel(time = im_date, method = 'nearest')
    ds = full_ds.sel(time = im_date, method = 'nearest')


    r, b, mae, rmse = get_stats(ds['snow_depth'], ds['lidar-sd'], nrmse = False)
    rand_r, rand_b, rand_mae, rand_rmse = get_stats(r_ds['snow_depth'], r_ds['lidar-sd'], nrmse = False)
    fischer_z = fischerz(ds['lidar-sd'].data.ravel(), ds['snow_depth'].data.ravel(), r_ds['snow_depth'].data.ravel())

    res.loc[site_name, 'random_r'] = rand_r
    res.loc[site_name, 'random_rmse'] = rand_rmse
    res.loc[site_name, 'fischerz'] = fischer_z
    res.loc[site_name, 'real_r'] = r
    res.loc[site_name, 'real_rmse'] = rmse
    
    rands.append(r_ds['snow_depth'].data.ravel())
    s1.append(ds['snow_depth'].data.ravel())
    sd.append(ds['lidar-sd'].data.ravel())

    res.to_csv(out_dir.joinpath('synthetic_v1.csv'))
    print(site_name + ' done')

rands, sd, s1 = np.concatenate(rands), np.concatenate(sd), np.concatenate(s1)

rand_r, rand_b, rand_mae, rand_rmse = get_stats(rands, sd)
r, b, mae, rmse = get_stats(s1, sd)

res.loc['All Sites', 'random_r'] = rand_r
res.loc['All Sites', 'random_rmse'] = rand_rmse
res.loc['All Sites', 'real_r'] = r
res.loc['All Sites', 'real_rmse'] = rmse
res.loc['All Sites', 'fischerz'] = fischerz(sd, s1, rands)
res.to_csv(out_dir.joinpath('synthetic_v1.csv'))

Mores 2020 random created
Mores 2020 done
Fraser 2021 random created
Fraser 2021 done
Dry Creek 2020 random created
Dry Creek 2020 done


# adding increasing gaussian noise to s1 vv and vh

# reordering VV and VH images